In [88]:
pip install galois

In [89]:
# Needed libraries
import secrets
import math
import numpy as np
import hashlib
import galois

# Generate private seed

In [90]:
#Generate Private Seed
def generate_private_seed()->bytes:
  return secrets.token_bytes(32)

# SHAKE A.K.A function H (Squeeze public_seed and T)

## SHAKE using hashlib

In [91]:
def create_private_sponge(private_seed:bytes,v:int,m:int,lvl)->bytes:
  # USE SHAKE
  if(lvl==1):
    h_shake = hashlib.shake_128(private_seed)
  else:
    h_shake = hashlib.shake_256(private_seed)
  # Private Sponge
  return h_shake.digest(32+math.ceil(m/8)*v)

#Extract public seed

In [92]:
def get_public_seed(private_sponge:bytes)->bytes:
  return private_sponge[:32]

#Extract T

In [93]:
def int8_to_binString(intByte:int)->str:
  """Get the 8 bit representation of an integer as a binary string
     without the '0b' indicator.

  The function takes one integer byte, turns it to binary
  and returns an string containing the 8 bit representation of it.

  Parameters
  ----------
  intByte : int
    The byte, labeled as an int by python.

  Returns
  -------
  str
      A string with 8 bits to represent the integer.
  """
  binary = bin(intByte)[2:]
  return binary.zfill(8)

In [94]:
def int8_to_bits(intByte:int)->list:
  """Get the first 8 bits of an integer as an array.

  The function takes one integer byte, extracts it's bits
  and returns an array containing the bits starting from
  the most significant bit.

  Parameters
  ----------
  intByte : int
    The byte, labeled as an int by python.

  Returns
  -------
  list
      An array with the first 8 bits of the integer
      starting from the most significant one.
  """
  bits_data = bin(intByte)[2:]
  return [int(i) for i in bits_data.zfill(8 * ((len(bits_data) + 7) // 8))]

In [95]:
def tMatrixRowGen(byteString:bytes,neededBits:int)->list:
  """Generate one row of the T matrix.

  The function takes one byte-string and extracts the specified
  ammount of bits (neededBits) to fill one row. When the ammount
  of bits doesn't fit completely in a byte, the most significant
  bits of the last byte are ignored.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the row.
  neededBits : int
    The ammount of bits a row needs to have.

  Returns
  -------
  list
      An array representing one row of the T matrix
  """
  row = []
  while(neededBits>8):
    row+=int8_to_bits(byteString[0])
    byteString = byteString[1:]
    neededBits = neededBits-8
  if(neededBits>0):
    row+=int8_to_bits(byteString[0])[-neededBits:]
  return row

def tMatrixGenerator(byteString:bytes,v:int,m:int)->np.ndarray:
  """Generate the T matrix.

  The function takes one byte-string and generates v rows
  each one with m bits extracted sequentially from a byteString
  using the tMatrixRowGen() function.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the matrix.
  v : int
    The ammount rows of the matrix.
  m : int
    The ammount columns of the matrix.

  Returns
  -------
  list
      An array representing the T matrix
  """
  mdivided8 =math.ceil(m/8)
  matrix = []
  for i in range(v):
    matrix.append(tMatrixRowGen(byteString,m))
    byteString = byteString[mdivided8:]
  return np.array(matrix)

# SHAKE (squeeze public map)

In [96]:
def subMatrixGenerator(byteString:bytes)->np.ndarray:
  """Generate a part of the L or Q1 matrix.

  The function takes one byte-string and returns one 16 row part
  of the L or Q1 matrix. It must be noted that the rows are generated as columns
  at first and then are transposed.

  Parameters
  ----------
  byteString : bytes
    A byte-string containing the bits used to fill the matrix.

  Returns
  -------
  np.ndarray
      A matrix
  """

  matrix = [int8_to_bits(byteString[i*2])+int8_to_bits(byteString[i*2+1]) for i in range(len(byteString)//2)]

  return np.array(matrix).T

## Obtain C, L, and $$Q_1$$

In [97]:
def G(v:int,m:int,lvl:int,public_seed:bytes)->tuple[np.ndarray,np.ndarray,np.ndarray]:
  n = m+v
  g_equation = 2+2*n+v*(v+1)+2*v*m
  mdivided16 = math.ceil(m/16)
  #first g function iteration
  if(lvl==1):
    g_shake = hashlib.shake_128(public_seed+bytes.fromhex('00'))
  else:
    g_shake = hashlib.shake_256(public_seed+bytes.fromhex('00'))
  public_sponge = g_shake.digest(g_equation)
  C_base=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
  public_sponge = public_sponge[2:]
  # L and Q1 have more than one dimension, so they are initilized differently
  L_base = subMatrixGenerator(public_sponge[:2*n])
  public_sponge = public_sponge[2*n:]
  Q1_base = subMatrixGenerator(public_sponge)

  #following g function iterations
  if(lvl==1):
    for i in range(1,mdivided16):
      g_shake = hashlib.shake_128(public_seed+bytes.fromhex(f'0{i}'))
      public_sponge = g_shake.digest(g_equation)
      C_base+=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
      public_sponge = public_sponge[2:]
      L_base = np.concatenate((L_base,subMatrixGenerator(public_sponge[:2*n])))
      public_sponge = public_sponge[2*n:]
      Q1_base = np.concatenate((Q1_base,subMatrixGenerator(public_sponge)))
  else:
    for i in range(1,mdivided16):
      g_shake = hashlib.shake_256(public_seed+bytes.fromhex(f'0{i}'))
      public_sponge = g_shake.digest(g_equation)
      C_base+=int8_to_bits(public_sponge[0])+int8_to_bits(public_sponge[1])
      public_sponge = public_sponge[2:]
      L_base = np.concatenate((L_base,subMatrixGenerator(public_sponge[:2*n])))
      public_sponge = public_sponge[2*n:]
      Q1_base = np.concatenate((Q1_base,subMatrixGenerator(public_sponge)))

  # obtaining C: we take the m bits (starting from the last generated bit) we
  # need fromthe total of bits generated
  C = C_base[-m:]
  C = np.array(C)
  # obtaining L: We discard the upper rows to get the specified dimensions
  L = L_base[-m:]
  # obtaining Q_1: We discard the upper row to get the specified dimensions
  Q1=Q1_base[-m:]
  return C,L,Q1


## Obtaining $$Q_2$$

In [98]:
def findPk1(v:int,m:int,k:int,q1:np.ndarray)->np.ndarray:
  """An implementation of the findPk1 algorithm of the LUOV specification.

  Parameters
  ----------
  v : int
    The v parameter of the LUOV specification.
  m : int
    The m parameter of the LUOV specification.
  k : int
    An integer between 1 and m.
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.

  Returns
  -------
  np.ndarray
      The v-by-v matrix representing the part of pk that is
      quadratic in the vinegar variables.
  """
  Pk1 = np.zeros([v,v],dtype=np.int8)
  column = 0
  for i in range(v):
    for j in range(i,v):
      Pk1[i,j]= q1[k,column]
      column+=1
    column+=m
  return Pk1

In [99]:
def findPk2(v:int,m:int,k:int,q1:np.ndarray)->np.ndarray:
  """An implementation of the findPk2 algorithm of the LUOV specification.

  Parameters
  ----------
  v : int
    The v parameter of the LUOV specification.
  m : int
    The m parameter of the LUOV specification.
  k : int
    An integer between 1 and m.
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.

  Returns
  -------
  np.ndarray
      The v-by-m matrix representing the part of pk that is bilinear in the
      vinegar variables and the oil variables.
  """
  Pk2 = np.zeros([v,m],dtype=np.int8)
  column = 0
  for i in range(v):
    column += v-i
    for j in range(m):
      Pk2[i,j]= q1[k,column]
      column+=1
  return Pk2

In [100]:
def findQ2(v:int,m:int,Q1:np.ndarray,T:np.ndarray)->galois.GF2:
  """An implementation of the findQ2 algorithm of the LUOV specification.

  Parameters
  ----------
  v : int
    The v parameter of the LUOV specification.
  m : int
    The m parameter of the LUOV specification.
  q1 : np.ndarray
    First part of Macaulay matrix of the quadratic part of P.
  t : np.ndarray
    A v-by-m matrix.

  Returns
  -------
  np.ndarray
      The second part of Macaulay matrix for quadratic part of P.
  """
  # we use galois to prevent the results of the operations from
  # being out of the finite field
  GF = galois.GF(2)
  T = GF(T)

  Q2 = GF(np.zeros([m,int(m*(m+1)/2)],dtype=np.int8))
  for k in range(m):
    Pk1 = GF(findPk1(v,m,k,Q1))
    Pk2 = GF(findPk2(v,m,k,Q1))
    Pk3 = -np.transpose(T)@Pk1@T+np.transpose(T)@Pk2
    column = 0
    for i in range(m):
      Q2[k,column]= Pk3[i,i]
      column += 1
      for j in range(i+1,m):
        Q2[k,column] = Pk3[i,j] + Pk3[j,i]
        column+=1
  return Q2

# Generate the public key

In [101]:
def keyGen(v:int,m:int,lvl:int,private_seed:bytes)->bytes:
  private_sponge = create_private_sponge(private_seed,v,m,lvl)
  public_seed = get_public_seed(private_sponge)

  T_base = private_sponge[32:]
  T = tMatrixGenerator(T_base,v,m)

  C,L,Q1 = G(v,m,lvl,public_seed)
  Q2 = findQ2(v,m,Q1,T)
  Q2_copy = Q2.copy()
  # The first 32 bytes of the public key come from the public seed.
  public_key = public_seed

  # Enconding of Q2
  Q2_bytes = b""
  # Get an array that contains all the elements of Q2 iterating column-wise.
  Q2 = Q2.flatten(order="F")
  while len(Q2)>8:
    # Take 8 elements of the Q2 flattened array, reverse their order, turn them
    # into a byte and add them to the bytes representing Q2.
    byte = str(Q2[7])+str(Q2[6])+str(Q2[5])+str(Q2[4])+str(Q2[3])+str(Q2[2])+str(Q2[1])+str(Q2[0])
    Q2_bytes += int(byte,2).to_bytes(1,byteorder="big")
    Q2 = Q2[8:]
  if len(Q2)>0:
    byte = ""
    while len(Q2)>0:
      byte += str(Q2[0])
      Q2 = Q2[1:]
    # Pad with zeros the remaining elements to encode, reverse their order, turn
    # them into a byte and add them to the bytes representing Q2.
    byte = byte.ljust(8,'0')[::-1]
    Q2_bytes += int(byte,2).to_bytes(1,byteorder="big")
  # Add the byte encoding of Q2 to the public key.
  public_key += Q2_bytes

  return public_key

In [102]:
private_seed_test = generate_private_seed()
public_key_test = keyGen(197,57,1,private_seed_test)
print(public_key_test)
print(len(public_key_test))

b'h&\xec\r~ uH\xee?\xed\x07P+\x94\xa5P\xb5;B\xbb\x15\xba\xe6\xc9\x14\xfa\x14\x11\x18x\xbexc\x03\xea8\x0e\x1b\x88OU\x90\xf0W{\xcf\xb5\xea~V\xca\xd7,S\x1c}\x96T6\xbc\x04DJ\xf0\xb5q}\xbd\x1e\xdaq\x94\xdc|\xad\n\xa7\xbe\xeaAF\x15Q_\xf3\xf7\x0b\x08\x01O\x86\x86kW\xe4\xe2\x98v\xe7^J\xf1\x14\ra\xe7\xc2\x11\xb3\xa8\xf80\xec\xef>lx\xa7z\xc3K\x99\x85\xb5\x9f";\xd6\x02V\xae\x82\xd1$\xe9\x80oF\xc9\xf3\xc4\xe0UN\xb6;\x9e\x9a0I\xb9qA\xc0-\x1e\xdb\x00N\xe4yR\xcf\x07\x17\xbff r\xe4rW\x08s^m\xf2\n\x12\x1b\x1fX\xe7\xb4\x8f\xeb\xden\xe6\x8e\x96D%\x8ew\xaa}\xc0n\x9b\xbfP\xef\x02\x8f\xd0\x94*\xadS\xdb~\x01A\xf06fR~N\xb8\xb6\xdfF\xd9.3c\x1d&\x7f\xd5\x0f:\xcf\x01{\x02\x08\x82\xc4$\xf7\xa0|e\xa7\xdb\xf8\xae\xfa(\x06~\xb3M\xf3\x8c\xb8N\x08Qi\x98_\x179\x9e\x8c+\xaa$bCLn;\xd84\x83\x0c9?\x8c\x8b\x01\xa8W\x06\x98~8\n\xc7m\xef\xb1Z\rT\xb5\xd2\xb7\xbe\x14\xb5\x88\x14U\xf7|\x0c\xf2\xeam\x1e\n\xd3\xdc]D\xd1E\xafGy-\xb7\xc4?\xe4RR\xd4;u\xc4\xd0/\xe0\xc5\xae\x02\x8a\x91\xbf\xd8!R\x1fR\x12\x8e\x98\xc9n\x1d\xa5\xa8\xf4\xe

# Sign

## Generate Salt

In [103]:
def generateSalt()->bytes:
  return secrets.token_bytes(16)

## hash digest

In [104]:
def generate_hash_digest(message:str,salt:bytes,m:int,r:int,lvl:int)->bytes:
  """Generate the hash digest (h of the LUOV specification).

  Parameters
  ----------
  message : str
    The message.

  salt : bytes
    The salt.

  m : int
    The m paramter of the LUOV specification.

  r : int
    The r paramter of the LUOV specification.

  lvl : int
    The lvl paramter of the LUOV specification.

  Returns
  -------
  bytes
      h (the bytes digested for the h variable of the LUOV specification).
  """
  if lvl == 1:
    h_digest_shake = hashlib.shake_128(message.encode("utf-8")+b'\x00'+salt)
  else:
    h_digest_shake = hashlib.shake_256(message.encode("utf-8")+b'\x00'+salt)

  h_digest_bytes = h_digest_shake.digest(math.ceil(m*r/8))

  # Join all the bytes into a single bit-string representation.
  h_digest_bits = ''.join([int8_to_binString(i) for i in h_digest_bytes])

  # Take r bits sequentialy from the string m times, to create a vector within the F(2**r) field
  # The bits that remain after this are discarded.
  h_digest = [int(h_digest_bits[i*r:i*r+r],base=2) for i in range(m)]

  return h_digest


## Generate v

In [105]:
def generateV(r:int, v:int)->list:
  """Generate the vinegar variables vector.

  The function generates the closest amount of bytes to contain r*v bits in order
  to be able to get a vector of size v, with each element having r bits.

  Parameters
  ----------
  r : int
    The r paramter of the LUOV specification.

  v : int
    The v paramter of the LUOV specification.

  Returns
  -------
  list
      A list that represents the vinegar variables.
  """
  vinegar_bytes = secrets.token_bytes(math.ceil(r*v/8))

  # Join all the bytes into a single bit-string representation.
  vinegar_bits = ''.join([int8_to_binString(i) for i in vinegar_bytes])

  # Take r bits sequentialy from the string v times, to create a vector within the F(2**r) field
  # The bits that remain after this are discarded.
  vinegar = [int(vinegar_bits[i*r:i*r+r],base=2) for i in range(v)]
  return vinegar

## Build augmented matrix

In [106]:
def buildAugmentedMatrix(C:galois.FieldArray,
                         L:galois.FieldArray,
                         Q1:galois.FieldArray,
                         T:galois.FieldArray,
                         h:galois.FieldArray,
                         v_array:galois.FieldArray,
                         m:int,v:int,r:int
                         )->tuple[galois.FieldArray,galois.FieldArray]:
  """The implementation of the buildAugmentedMatrix algorithm of the LUOV specification.

  Parameters
  ----------
  C : galois.FieldArray
    The C vector of the LUOV specification.

  L : galois.FieldArray
    The L matrix of the LUOV specification.

  Q1 : galois.FieldArray
    The Q1 matrix of the LUOV specification.

  T : galois.FieldArray
    The T matrix of the LUOV specification.

  h : galois.FieldArray
    The hash digest to target.

  v_array : galois.FieldArray
    A vector containing the vinegar variables.

  m : int
    The m paramter of the LUOV specification.

  v : int
    The v paramter of the LUOV specification.

  r : int
    The r paramter of the LUOV specification.

  Returns
  -------
  (galois.FieldArray,galois.FieldArray)
      The arrays representing the augmented matrix.
      The first being the left part and the second being the right part.
  """
  n = m+v
  v_with_zeros = v_array.copy()
  v_with_zeros.resize(n)
  RHS = h - C-L@v_with_zeros
  T_with_ones = np.concatenate((-T,np.identity(m)),axis=0)
  LHS = L@T_with_ones

  GF = galois.GF(2**r,irreducible_poly = 0x83)

  for k in range(m):
    Pk1=GF(findPk1(v,m,k,Q1))
    Pk2=GF(findPk2(v,m,k,Q1))
    RHS[k] = RHS[k] - v_array.T@Pk1@v_array

    Fk2=-(Pk1+Pk1.T)@T+Pk2

    LHS[k] = LHS[k] + v_array@Fk2
  # Here the implementation differs from the specification,
  # instead of returning the augmented matrix LHS and RHS are
  # returned separately to use numpy's solving function
  return LHS,RHS


## Sign

In [107]:
def encode_field_element(element:galois.typing.DTypeLike,r:int)->str:
  """Get the r bit representation of an field element as a binary string
     without the '0b' indicator.

  The function takes one field element, turns it to binary
  and returns an string containing the r bit representation of it.

  Parameters
  ----------
  element : galois.typing.DTypeLike
    The field element (The number if you prefer).

  r : int
    The ammount of bits required to encode the element.

  Returns
  -------
  str
      A string with r bits to represent the field element.
  """
  return bin(element)[2:].zfill(r)

In [108]:
def encode_siganture(signature:galois.FieldArray,salt:bytes,n:int,r:int)->bytes:
  """Encoding of the signature with the salt following the LUOV specification.

  The function takes a FieldArray, converts all its elements to binary and
  concatenates them in order, then it encodes the result into bytes (padding with
  zeros at the end to achieve the number of bytes). Finaly, the result is concatenated
  with the salt.

  Parameters
  ----------
  signature : galois.FieldArray
    The FieldArray containing the elements of the signature.

  salt : bytes
    The salt that will be added at the end of the signature.

  n : int
    The number of elements in the signature.

  r : int
    The ammount of bits of each element of the signature.

  Returns
  -------
  bytes
      The bytes of the signature and the salt concatenated.
  """
  binary_signature = ''
  # Turn all the elements in the signature vecto to an r-bit per element binary
  # string.
  for i in signature:
    binary_signature += encode_field_element(i,r)
  needed_bits = n*r
  byte_signature = b''
  # Iterate through the binary string, taking 8 bits at a time to get the bytes
  # of the signature.
  while needed_bits>8:
    byte_signature += int(binary_signature[:8],2).to_bytes(1,byteorder='big')
    binary_signature = binary_signature[8:]
    needed_bits -= 8
  # The remaining bits are padded with zeros at the end to make the last byte.
  if needed_bits>0:
    for i in range(8-needed_bits):
      binary_signature += '0'
    byte_signature += int(binary_signature,2).to_bytes(1,byteorder='big')
  return byte_signature + salt

In [109]:
def sign(message:str,private_seed:bytes,v:int,m:int,r:int,lvl:int)->bytes:
  """The sign algorithm of the LUOV specification.

  The function takes a private seed and a message. Then, given the parameters of
  the LUOV specification, it generates a signature.

  Parameters
  ----------
  message : str
    The message to be signed.

  private_seed : bytes
    The private seed of the LUOV specification.

  v : int
    The v parameter of the LUOV specification.

  m : int
    The m parameter of the LUOV specification.

  r : int
    The r parameter of the LUOV specification.

  lvl : int
    The level of the LUOV specification.

  Returns
  -------
  bytes
      The signature for the given message.
  """
  private_sponge = create_private_sponge(private_seed,v,m,lvl)
  public_seed = get_public_seed(private_sponge)
  T_base = private_sponge[32:]
  T = tMatrixGenerator(T_base,v,m)

  GF = galois.GF(2**r,irreducible_poly = 0x83)

  T = GF(T)

  C,L,Q1 = G(v,m,lvl,public_seed)

  C = GF(C)
  L = GF(L)
  Q1 = GF(Q1)

  salt = generateSalt()
  h = GF(generate_hash_digest(message,salt,m,r,lvl))

  n = m+v
  no_solution = True
  s_prime = np.zeros([m,1])
  while no_solution:
    vinegar = GF(generateV(r=r,v=v))
    A0,A1= buildAugmentedMatrix(C,L,Q1,T,h,vinegar,m,v,r)

    try:
      # Try to solve the equation system given the vinegar variables
      oil = np.linalg.solve(A0,A1)
      # If we find a solution we assume it's unique and end the loop.
      no_solution = False
      # The solution (oil) is concatenated to de vinegar variables vector.
      s_prime = np.concatenate([vinegar,oil])
    except np.linalg.LinAlgError:
      # If no solution is found we stay in the loop to try with new
      # vinegar variables
      no_solution = True

  # Build the matrix for the solution operation
  solution_operand_left = np.concatenate([np.identity(v),np.zeros([m,v])],axis=0)
  solution_operand_right = np.concatenate([-T,np.identity(m)],axis=0)
  solution_operand = np.concatenate([solution_operand_left,solution_operand_right],axis=1)

  # Build S using the matrix solution_operand
  s = GF(solution_operand)@s_prime
  encoded_signature=encode_siganture(s,salt,n,r)
  return encoded_signature

In [110]:
test_sign = sign("hi",private_seed_test,197,57,7,1)
print(test_sign)
print(len(test_sign))

b'\x92\xa3f3\xfcE\xe8\xc4\x96\rJ>\t\x19\xff\xe7)\x8fj\xf6\xfe\xc7?\xc2O\xc3\xfaY\x98\x0c\x9b\x85@\x8e\x8c?\'\xb7\x1d2\x14N)\xdc[\xc6o\xee\x03\xf5\x1f\xc3\x10Q\xf1\x14\x122\xc7\xaf\xcc\x072\\\xf6\x13K\xe4C\xde\xc0\x81\xb4\x88\x83\x8b\xac\x1c\xad\xcf\x9eU1\xe1\xcf\xa6z\x9d\x94W\x0f\x8c\xfd\xcdS\xceE\xa3s\xf2\xf5!\xf1I8\xe0\xc9\xba\xa2^\xa0&s\xe1\xefd\xd4\x9e\xaaB\xf2\x9fD\x8e"\x15\xb0\x05\xeb\xaaJ\x87 Y\xcas\xfbS\xb4-G\x0e\xa9J\xcf\xf8\xc2\xf9\xae\xf8(\xfa\xd8\xee\xb0x\xde\xec\xe6\xe6\xcc\xe3.\x95\x82\xdby4u\xe0\xeal\xfc\xaa\xe0\xcdi\x04\xbb\x1b\xd3\x95\xd2\r#\x14`1\xd8\x91d1\xbe\xffy\xaa\xd0\xd9$Z\xee\xd5A\xa4\xbe\xf3\x07<MMk\x1f\xfd;\x90j\xab\xdd\xd2\xe7\x0f\xbe\xc0h?Y\xc34O\xaa\x91\x83\x03j\x934\x0f\xd3\xfc'
239


# Verify

In [111]:
def extract_seed_and_q2(public_key:bytes,m:int)->tuple[bytes,np.ndarray]:
  """Extract the public seed and Q2 from a public key.

  The function takes a public key and extracts the public seed from it's first
  32 bytes. The remaining bytes are interpreted as the columns of the Q2 matrix.

  Parameters
  ----------
  public_key : bytes
    A LUOV specification public key.

  m : int
    The m parameter of the LUOV specification.

  Returns
  -------
  bytes
      The public seed of the LUOV specification contained in the public key.

  np.ndarray
      The Q2 matrix of the LUOV specification contained in the public key.
  """
  # Get the first 32 bytes that belong to the public seed.
  public_seed = public_key[:32]

  # The remaining bytes are the encoded elements of Q2.
  Q2_bytes = public_key[32:]
  Q2_bits = []
  for i in Q2_bytes:
    # The bits are extracted from the bytes.
    reversed =int8_to_bits(i)
    # The extracted bits are reversed back to their original value and
    # concatenated in a list containing all the bits of Q2.
    reversed.reverse()
    Q2_bits+= reversed
  #The padding zeros for the encoding of Q2 are discarded.
  Q2_bits = Q2_bits[:int(m*(m*(m+1)/2))]
  Q2 = np.array(Q2_bits)
  # The array is reshaped to form the original Q2 matrix, column by column.
  Q2 = np.reshape(Q2,(m,int(m*(m+1)/2)),order="F")
  return public_seed,Q2

In [112]:
def decode_signature(signature:bytes,r:int)->np.ndarray:
  """Decode a signature.

  The function takes a signature and discards the salt from it. Then it
  extracts all the bits from the bytes of the signature and joins them in
  a single r*m long string. Finally it takes the bits in r-sized chunks to
  get each element of the Array of the field F_(2**r) with m elements.

  Parameters
  ----------
  signature : bytes
    A LUOV specification signature.

  r : int
    The r parameter of the LUOV specification.

  Returns
  -------
  np.ndarray
      The array contained in a signature of the LUOV specification.
  """
  # Discard the salt.
  signature_without_salt = signature[:-16]
  signature_bits = ''
  # Get all the bits that form the signature's array
  for i in signature_without_salt:
    signature_bits+=int8_to_binString(i)
  signature_array = []
  while len(signature_bits)>r:
    # Get r bits from the signature_bit's string and interpret them as an
    # element of the original array of the signature which has m elements of the
    # F_(2**r) finite field.
    signature_array.append(int(signature_bits[:r],2))
    signature_bits = signature_bits[r:]
  return np.array(signature_array)

In [113]:
def get_salt(signature:bytes)->bytes:
  """Extract the salt from a signature.

  The function takes a signature, then it extracts the salt from it, which is
  contained in it's last 16 bytes.

  Parameters
  ----------
  signature : bytes
    A LUOV specification signature.

  Returns
  -------
  bytes
      The salt of a LUOV signature.
  """
  return signature[-16:]

In [114]:
def evaluatePublicMap(public_key:bytes,s:bytes,v:int,m:int,r:int,lvl:int)->galois.FieldArray:
  """The evaluatePublicMap algorithm of the LUOV specification.

  The function takes a public key and a signature, then, given the
  parameters of the LUOV specification used for both the key and signature, it
  evaluates the public map e.

  Parameters
  ----------
  public_key : bytes
    The public key of the signer.

  s : bytes
    A signature.

  v : int
    The v parameter of the LUOV specification.

  m : int
    The m parameter of the LUOV specification.

  r : int
    The r parameter of the LUOV specification.

  lvl : int
    The level of the LUOV specification.

  Returns
  -------
  galois.FieldArray
      The evaluation of the public map.
  """
  public_seed,Q2 = extract_seed_and_q2(public_key,m)
  GF = galois.GF(2**r,irreducible_poly = 0x83)
  C,L,Q1 = G(v,m,lvl,public_seed)

  C = GF(C)
  L = GF(L)
  Q1 = GF(Q1)
  Q = np.concatenate([Q1,GF(Q2)],axis=1)

  s_decoded = GF(decode_signature(s,r))

  e = C + L@s_decoded
  column = 0
  n= m+v
  for i in range(n):
    for j in range(i,n):
      for k in range(m):
        e[k] = e[k]+Q[k,column]*s_decoded[i]*s_decoded[j]
      column = column + 1
  return e

In [115]:
def verify(public_key:bytes,message:str,candidate_signature:bytes,v:int,m:int,r:int,lvl:int)->bool:
  """Verification of a signed message

  The function takes a public key, a message and a signature, then, given the
  parameters of the LUOV specification used for both the key and signature, it
  evaluates if the signature is valid.

  Parameters
  ----------
  public_key : bytes
    The public key of the signer.

  message : str
    The message signed message.

  candidate_signature : bytes
    The signature of the message.

  v : int
    The v parameter of the LUOV specification.

  m : int
    The m parameter of the LUOV specification.

  r : int
    The r parameter of the LUOV specification.

  lvl : int
    The level of the LUOV specification.

  Returns
  -------
  bool
      The verification result (True if the signature for the message is valid).
  """
  GF = galois.GF(2**r,irreducible_poly = 0x83)
  h = GF(generate_hash_digest(message,get_salt(candidate_signature),m,r,lvl))
  e = evaluatePublicMap(public_key,candidate_signature,v,m,r,lvl)
  return np.array_equal(h,e)

In [116]:
print(verify(public_key_test,"hi",test_sign,197,57,7,1))

True
